# AI 500 - PA1 Minimax

<p align="center">
  <img src="https://www.shutterstock.com/image-photo/game-4-row-yellow-red-600nw-2233654429.jpg" alt="Connect 4" />
</p>


For this assignment you will be attempting to implement the minimax algorithm for the game Connect 4. The game is played on a 6x7 board where two players take turns dropping their pieces into the columns. The first player to get 4 of their pieces in a row (horizontally, vertically, or diagonally) wins the game. If the board fills up before either player wins, the game is a draw.

## Import Libraries

In [1]:
# Uncomment the lines of code below to install the required libraries

%pip install math
%pip install termcolor

import math
from termcolor import colored  # For coloring the printed board

Note: you may need to restart the kernel to use updated packages.


ERROR: Could not find a version that satisfies the requirement math (from versions: none)
ERROR: No matching distribution found for math


## Required Code

The following functions help manage the game state and are required for the minimax algorithm to work. Please go through these functions to understand the game logic and mechanics.



### Constants and Function for the game setup

**Do NOT Alter This Code!**


In [2]:
#board dimensions
ROWS = 6
COLS = 7

PLAYER = 1  # player
AI = 2  # computer

EMPTY = 0  # Empty cell in the board
WINNING_LENGTH = 4  # Number of pieces in a row needed to win

# Function to create an empty board, initialising each empty spot as 0
def create_board():
    board = []
    for _ in range(ROWS):
        row = []
        for _ in range(COLS):
            row.append(EMPTY)
        board.append(row)
    return board

## Checking for winning condition (TO DO)

This function checks if the current player has won the game. It returns `True` if the player has won, otherwise it returns `False`. Remember, the winning condition is to have 4 pieces in a row (horizontally, vertically, or diagonally).

<span style="color: green;">You Must Implement This Function</span>


In [3]:
# Function to check if the current move is a winning move
def winning_move(board, piece):
    
    # checking for four adjacent pieces in a horizontal line
    for row in range(ROWS):
        for col in range(COLS - WINNING_LENGTH + 1):
            if all(board[row][col + i] == piece for i in range(WINNING_LENGTH)):
                return True
    
    # checking for four adjacent pieces in a vertical line
    for col in range(COLS):
        for row in range(ROWS - WINNING_LENGTH + 1):
            if all(board[row + i][col] == piece for i in range(WINNING_LENGTH)):
                return True
    
    # checking for four adjacent pieces in a rising diagonal line
    for row in range(ROWS - WINNING_LENGTH + 1):
        for col in range(COLS - WINNING_LENGTH + 1):
            if all(board[row + i][col + i] == piece for i in range(WINNING_LENGTH)):
                return True
    
    # checking for four adjacent pieces in a decreasing diagonal line
    for row in range(ROWS - 1, WINNING_LENGTH - 2, -1):
        for col in range(COLS - WINNING_LENGTH + 1):
            if all(board[row - i][col + i] == piece for i in range(WINNING_LENGTH)):
                return True
            
    return False


### Functions for managing the game state

These functions initialise the game board, check if a move is valid, and other related steps

In [4]:
# Function to create an empty board, initialising each empty spot as 0
def create_board():
    board = []
    for _ in range(ROWS):
        row = []
        for _ in range(COLS):
            row.append(EMPTY)
        board.append(row)
    return board

# Function to check if the selected column is a valid move
def is_valid_move(board, col):
    # A move is valid if the top cell of the column is empty
    return board[0][col] == EMPTY

# Function to get the next open row in a column
def get_next_open_row(board, col):
    # Starting from the bottom, find the first empty row in the selected column
    for r in range(ROWS - 1, -1, -1):
        if board[r][col] == EMPTY:
            return r

# Function to drop a piece into the board at the specified row and column
def drop_piece(board, row, col, piece):
    board[row][col] = piece

# Function to evaluate the score of a specific window (subsection) of the board
def evaluate_window(window, piece):
    score = 0
    opp_piece = PLAYER if piece == AI else AI

    # Assign scores based on how many pieces the current player has in this window
    if window.count(piece) == 4:
        score += 100
    elif window.count(piece) == 3 and window.count(EMPTY) == 1:
        score += 5
    elif window.count(piece) == 2 and window.count(EMPTY) == 2:
        score += 2

    # Penalize if the opponent has three pieces in a window
    if window.count(opp_piece) == 3 and window.count(EMPTY) == 1:
        score -= 4

    return score

# Function to calculate the total score of the board for the AI
def score_position(board, piece):
    score = 0

    # Score center column higher since it's more strategic
    center_array = [board[r][COLS // 2] for r in range(ROWS)]
    center_count = center_array.count(piece)
    score += center_count * 3

    # Score rows
    for r in range(ROWS):
        row_array = [board[r][c] for c in range(COLS)]
        for c in range(COLS - 3):
            window = row_array[c:c + WINNING_LENGTH]
            score += evaluate_window(window, piece)

    # Score columns
    for c in range(COLS):
        col_array = [board[r][c] for r in range(ROWS)]
        for r in range(ROWS - 3):
            window = col_array[r:r + WINNING_LENGTH]
            score += evaluate_window(window, piece)

    # Score positive diagonal
    for r in range(ROWS - 3):
        for c in range(COLS - 3):
            window = [board[r + i][c + i] for i in range(WINNING_LENGTH)]
            score += evaluate_window(window, piece)

    # Score negative diagonal
    for r in range(ROWS - 3):
        for c in range(COLS - 3):
            window = [board[r + 3 - i][c + i] for i in range(WINNING_LENGTH)]
            score += evaluate_window(window, piece)

    return score

# Function to get all valid moves (columns that are not full)
def get_valid_moves(board):
    return [c for c in range(COLS) if is_valid_move(board, c)]

# Function to check if the game is over (terminal state)
def is_terminal_node(board):
    return winning_move(board, PLAYER) or winning_move(board, AI) or len(get_valid_moves(board)) == 0

# Function to print the board, color-coded
def print_board(board):
    for row in board:
        colored_row = []
        for cell in row:
            if cell == PLAYER:
                colored_row.append(colored(str(cell), 'red'))  # Color Player's pieces red
            elif cell == AI:
                colored_row.append(colored(str(cell), 'green'))  # Color AI's pieces green
            else:
                colored_row.append(str(cell))
        print(" ".join(colored_row))
    print("\n")

## Minimax Function (TO DO!)

This function implements the minimax algorithm. It should return the best move for the current player and its score as a tuple `(score, move)` where `score` is the score of the best move and `move` is the column where the piece should be placed.


In [7]:
def minimax(board, depth, maximizingPlayer):
    # looking for the available possible moves
    moves = get_valid_moves(board)
    # is game over
    isterminal = is_terminal_node(board)

    if depth == 0 or isterminal:
        # game is over
        if isterminal:
            # AI wins
            if winning_move(board, AI):
                return (None, 100000)
            # Player wins
            elif winning_move(board, PLAYER):
                return (None, -100000)
            # Tie
            else:
                return (None, 0)
        else:
            # reached at depth = 0. to complete the recursive function, we return score
            return (None, score_position(board, AI))

    if maximizingPlayer:
        # -infinity is the lowest possible value, ensuring that any realistic score will be higher and will cause an update
        maxvalue = -float('inf') 
        bestcol = None
        
        for col in moves:
            # possible rows to drop piece in
            row = get_next_open_row(board, col)
            # creating a hypothetical board
            temp = [row[:] for row in board] 
            # dropping piece in the board for evaluation purposes
            drop_piece(temp, row, col, AI)  
            # getting max score for the move made
            new = minimax(temp, depth - 1, False)[1]

            if new > maxvalue:
                maxvalue = new
                bestcol = col

        return bestcol, maxvalue

    else:
        # infinity is the highest possible value, ensuring that any realistic score will be lower and will cause an update.
        minvalue = float('inf') 
        bestcol = None
        
        for col in moves:
            row = get_next_open_row(board, col)
            temp = [row[:] for row in board]  
            drop_piece(temp, row, col, PLAYER) 
            # getting the minimum score for the move made
            new = minimax(temp, depth - 1, True)[1]

            if new < minvalue:
                minvalue = new
                bestcol = col

        return bestcol, minvalue

## Main Function

This function executes the game loop. It calls the minimax function to get the best move for the current player and updates the game state accordingly. The game loop continues until the game is over.


In [8]:
# Main function to run the game
def main():
    board = create_board()  # Initialize the game board
    game_over = False
    turn = PLAYER  # Player 1 starts first

    while not game_over:
        if turn == PLAYER:
            # Human player's turn
            col = int(input("Player 1 Make your selection (0-6): "))
            if is_valid_move(board, col):
                row = get_next_open_row(board, col)
                drop_piece(board, row, col, PLAYER)

                # Check if the human player won
                if winning_move(board, PLAYER):
                    print("PLAYER 1 WINS!!")
                    game_over = True

        else:
            # computer turn
            col, minimax_score = minimax(board, 5, True)
            if is_valid_move(board, col):
                row = get_next_open_row(board, col)
                drop_piece(board, row, col, AI)

                # if computer won
                if winning_move(board, AI):
                    print("PLAYER 2 (AI) WINS!!")
                    game_over = True

        print_board(board)  # Print the current board state

        # Alternate turns
        turn = PLAYER if turn == AI else AI

        # Check for a draw
        if len(get_valid_moves(board)) == 0 and not game_over:
            print("It's a TIE!")
            game_over = True

if __name__ == "__main__":
    main()

Player 1 Make your selection (0-6): 5
0 0 0 0 0 0 0
0 0 0 0 0 0 0
0 0 0 0 0 0 0
0 0 0 0 0 0 0
0 0 0 0 0 0 0
0 0 0 0 0 1 0


0 0 0 0 0 0 0
0 0 0 0 0 0 0
0 0 0 0 0 0 0
0 0 0 0 0 0 0
0 0 0 0 0 0 0
0 0 0 2 0 1 0


Player 1 Make your selection (0-6): 5
0 0 0 0 0 0 0
0 0 0 0 0 0 0
0 0 0 0 0 0 0
0 0 0 0 0 0 0
0 0 0 0 0 1 0
0 0 0 2 0 1 0


0 0 0 0 0 0 0
0 0 0 0 0 0 0
0 0 0 0 0 0 0
0 0 0 0 0 0 0
0 0 0 2 0 1 0
0 0 0 2 0 1 0


Player 1 Make your selection (0-6): 5
0 0 0 0 0 0 0
0 0 0 0 0 0 0
0 0 0 0 0 0 0
0 0 0 0 0 1 0
0 0 0 2 0 1 0
0 0 0 2 0 1 0


0 0 0 0 0 0 0
0 0 0 0 0 0 0
0 0 0 0 0 2 0
0 0 0 0 0 1 0
0 0 0 2 0 1 0
0 0 0 2 0 1 0


Player 1 Make your selection (0-6): 5
0 0 0 0 0 0 0
0 0 0 0 0 1 0
0 0 0 0 0 2 0
0 0 0 0 0 1 0
0 0 0 2 0 1 0
0 0 0 2 0 1 0


0 0 0 0 0 0 0
0 0 0 0 0 1 0
0 0 0 0 0 2 0
0 0 0 0 0 1 0
0 0 0 2 0 1 0
0 0 2 2 0 1 0


Player 1 Make your selection (0-6): 5
0 0 0 0 0 1 0
0 0 0 0 0 1 0
0 0 0 0 0 2 0
0 0 0 0 0 1 0
0 0 0 2 0 1 0
0 0 2 2 0 1 0


0 0 0 0 0 1 0
0 0 0 0 0 1 0
0 0 0 0 